# Classification Exercise

## Import data

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


In [6]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [8]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [9]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [10]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### Perform a Train Test Split on the Data

In [11]:
from sklearn.model_selection import train_test_split

In [13]:
x_data = census.drop('income_bracket', axis=1)

In [14]:
y_labels = census['income_bracket']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

### Create the Feature Columns

In [16]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [17]:
import tensorflow as tf

In [18]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [19]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [20]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [23]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [24]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpw19hazyw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd001ae2048>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpw19hazyw/model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 141.545
INFO:tensorflow:loss = 759.015, step = 101 (0.711 sec)
INFO:tensorflow:global_step/sec: 186.178
INFO:tensorflow:loss = 161.064, step = 201 (0.540 sec)
INFO:tensorflow:global_step/sec: 207.167
INFO:tensorflow:loss = 256.908, step = 301 (0.478 sec)
INFO:tensorflow:global_step/sec: 230.72
INFO:tensorflow:loss = 231.499, step = 401 (0.434 sec)
INFO:tensorflow:global_step/sec: 239.545
INFO:tensorflow:loss = 114.609, step = 501 (0.418 sec)
INFO:tensorflow:global_step/sec: 196.139
INFO:tensorflow:loss = 48.359, step = 601 (0.508 sec)
INFO:tensorflow:global_step/sec: 229.912
INFO:tensorflow:loss = 42.0026, step = 701 (0.436 sec)
INFO:tensorflow:global_step/sec: 215.819
INFO:tensorflow:loss = 698.584, step = 801 (0.460 sec)
INFO:tensorflow:global_step/sec: 188.512
INFO:tensorflow:loss = 133.512, s

### Evaluation

In [26]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [27]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from /tmp/tmpw19hazyw/model.ckpt-5000


In [28]:
predictions[0]

{'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.303983], dtype=float32),
 'logits': array([-0.82840228], dtype=float32),
 'probabilities': array([ 0.69601697,  0.303983  ], dtype=float32)}

In [29]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.90      0.88      0.89      7436
          1       0.65      0.68      0.67      2333

avg / total       0.84      0.84      0.84      9769

